# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

In [ ]:
## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Back', 'Belly', 'Right', 'Stand', 'Knee', 'HeadBack', 'Crouch', 'Frog', 'Left', 'StandInit', 'Sit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)

print 'total number of data', len(all_data)

total number of data 222


In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:
# YOUR CODE HERE
clf.fit(np.asarray(all_data)[training_data], np.asarray(all_target)[training_data])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [8]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])

In [9]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [10]:
expected = []
predicted = []
# YOUR CODE HERE
expected = np.asarray(all_target)[training_data]
predicted = clf.predict(np.asarray(all_data)[training_data])

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        16
          1       1.00      1.00      1.00        11
          2       1.00      0.89      0.94         9
          3       1.00      1.00      1.00         9
          4       1.00      1.00      1.00         5
          5       1.00      1.00      1.00         7
          6       0.95      1.00      0.98        21
          7       1.00      1.00      1.00         8
          8       1.00      1.00      1.00        15
          9       1.00      1.00      1.00        36
         10       1.00      1.00      1.00        18

avg / total       0.99      0.99      0.99       155


Confusion matrix:
[[16  0  0  0  0  0  0  0  0  0  0]
 [ 0 11  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  1  0  0  0  0]
 [ 0  0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0  0  0]
 [ 0  0  0  0  0  0 21  0  0  0  0]
 [ 0  0  0  0 

## 4. Evaluate on the test data

In [11]:
expected = []
predicted = []
# YOUR CODE HERE
test_data = permutation[n_training_data:]
expected = np.asarray(all_target)[test_data]
predicted = clf.predict(np.asarray(all_data)[test_data])

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         7
          1       1.00      1.00      1.00         8
          2       1.00      1.00      1.00         2
          3       1.00      1.00      1.00         2
          4       1.00      1.00      1.00         5
          5       1.00      1.00      1.00         3
          6       1.00      1.00      1.00         9
          7       1.00      1.00      1.00         2
          8       1.00      1.00      1.00         5
          9       1.00      1.00      1.00        16
         10       1.00      1.00      1.00         8

avg / total       1.00      1.00      1.00        67


Confusion matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0]
 [ 0  8  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  9  0  0  0  0]
 [ 0  0  0  0 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [12]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [13]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])